In [1]:
using JuMP, Clp, Plots

In [15]:
N = 1
d = collect(60:N:150)
p = (1/size(d,1))*ones(size(d,1));
λ = 0.9
ω = 1.0

1.0

In [3]:
function Q(x,ξ)
    m = Model(Clp.Optimizer)
    set_optimizer_attribute(m, "LogLevel",0)
    @variables(m, 
    begin
        y ≥ 0
        w ≥ 0
    end)

    @constraints(m,
    begin
        y ≤ ξ
        aux, y + w ≤ x
    end)

    @objective(m, Min, -25 * y - 5 * w)
    optimize!(m)
    return objective_value(m), shadow_price(aux)
end

Q (generic function with 1 method)

In [4]:
function cvar(q_list)
    c = Model(Clp.Optimizer)
    set_optimizer_attribute(c, "LogLevel",0)
    @variables(c, 
    begin
        q[1:size(q_list,1)] ≥ 0
    end)

    @constraints(c,
    begin
        sum(q[s] for s = 1:size(q_list,1)) == 1
        aux[s=1:size(q_list,1)], q[s] ≤ (p[s]/(1-λ))
    end)

    @objective(c, Max, sum(q[s]*q_list[s] for s = 1:size(q_list,1)))
    optimize!(c)
    return value.(q)
end 

cvar (generic function with 1 method)

In [25]:
master = Model(Clp.Optimizer)
set_optimizer_attribute(master, "LogLevel",0)
@variable(master,x>=0)
@variable(master,α)
@constraint(master, α >= -9999)
@constraint(master, x<=150)
@objective(master, Min, 10*x + α)
optimize!(master);

iterator = 1
Q̂i_list = []
πi_list = []
x̂i_list = []
LB_list = []
UB_list = []

while true
    LB = objective_value(master)
    x̂i = value(x)
    append!(x̂i_list, x̂i)
    
    slave_Q̄i_list = []
    slave_πi_list = []
    for ξ in d
        q̂, pî = Q(x̂i,ξ)
        append!(slave_Q̄i_list, q̂)
        append!(slave_πi_list, pî)
    end
    
    cvar_results_Q̄i = cvar(slave_Q̄i_list)
    cvar_results_πi = cvar(slave_πi_list)
    
    Q̂i = sum(((1-ω) * p[s] + ω * cvar_results_Q̄i[s]) * Q(x̂i,d[s])[1] for s = 1:size(d,1))
    πi = sum(((1-ω) * p[s] + ω * cvar_results_πi[s]) * Q(x̂i,d[s])[2] for s = 1:size(d,1))


    l_i = x-> Q̂i + πi*(x-x̂i)
    append!(Q̂i_list, Q̂i)
    append!(πi_list, πi)
    
    UB = 10*x̂i + sum(((1-ω) * p[s] + ω * cvar_results_Q̄i[s]) * Q(x̂i,d[s])[1] for s = 1:size(d,1))  
    append!(LB_list, LB)
    append!(UB_list, UB)
        
    if trunc(LB, digits=5) == trunc(UB, digits=5)
        println("**SOLUTION FOUND**")
        println( "Iteration $(iterator): x̂$(iterator): $(x̂i) | Lower Bound: $(LB) & Uper Bound: $(UB)")
        println("===========")
        break
    else
        println( "Iteration $(iterator): x̂$(iterator): $(x̂i) | Lower Bound: $(LB) & Uper Bound: $(UB)")
        println("===========")
    end  
    @constraint(master, α >= l_i(x))
    optimize!(master);
    
    iterator += 1
end

Iteration 1: x̂1: 0.0 | Lower Bound: -9999.0 & Uper Bound: 0.0
Iteration 2: x̂2: 150.0 | Lower Bound: -2250.0 & Uper Bound: -531.098901098901
Iteration 3: x̂3: 64.05494505494507 | Lower Bound: -960.8241758241758 & Uper Bound: -938.2423620335709
Iteration 4: x̂4: 66.56097560975608 | Lower Bound: -948.2940230501208 & Uper Bound: -943.6303939962477
Iteration 5: x̂5: 65.50000000000001 | Lower Bound: -944.0384615384618 & Uper Bound: -942.9395604395605
**SOLUTION FOUND**
Iteration 6: x̂6: 66.00000000000007 | Lower Bound: -943.846153846154 & Uper Bound: -943.846153846154
